In [22]:
from elasticsearch import Elasticsearch, helpers
import csv
import json 
# Connexion au cluster Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Nom de l'index
index_name = "review3"

# Mapping pour l'index 
mapping = {
    "mappings": {
        "_meta": {
            "created_by": "yaya"
        },
        "properties": {
            "@timestamp": {"type": "date"},
            "Company": {"type": "keyword"},
            "Customer": {"type": "keyword"},
            "Date_experience": {"type": "date", "format": "yyyy-MM-dd"},
            "Date_reply": {"type": "date", "format": "yyyy-MM-dd"},
            "Date_review": {"type": "date", "format": "yyyy-MM-dd"},
            "Response_time":{"type": "long"},
            "Experience": {"type": "keyword"},
            "Language": {"type": "keyword"},
            "Number_review": {"type": "long"},
            "Rating": {"type": "long"},
            "Reply": {"type": "keyword"},
            "Status": {"type": "keyword"},
            "Title": {"type": "keyword"},
            "column1": {"type": "long"},
            "document_id": {"type": "keyword"}  # Ajout d'un champ pour l'ID du document
        }
    }
}

# Créer l'index avec le mapping (you can skip this if the index already exists)
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)

# Lecture du fichier CSV et correction des champs vides
with open("../data/processed/avis_clients.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    documents = []
    for idx, row in enumerate(reader):
        if "" in row:
            row["document_id"] = str(idx)  # Ajout de l'ID du document
            for key in list(row.keys()):
                if key == "":
                    row.pop(key)  # Suppression des champs avec des clés vides
        documents.append(row)

# Insérer les documents en vrac dans Elasticsearch
if documents:
    try:
        response = helpers.bulk(es, documents, index=index_name)
        print("Documents insérés avec succès :", response)
    except helpers.BulkIndexError as e:
        print("Erreur lors de l'insertion des documents :")
        for err in e.errors:
            print("Erreur :", err)

# Vérification des documents insérés
result = es.search(index=index_name, size=5)  # Récupère les 5 premiers documents

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])


C:\Users\yaya-\AppData\Local\Temp/ipykernel_4128/85386151.py:39: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index_name, body=mapping)


Documents insérés avec succès : (2515, [])


In [23]:
response = es.search(index="review3")
# Récupération du template
template = es.indices.get_mapping()

In [24]:
# recherche de tous les documents 
es = Elasticsearch(hosts="http://localhost:9200")

# Index des avis clients
index_name = "review3"

query = {
    "query": {
        "match_all": {}
    }
}

# Exécuter la requête
result = es.search(index="review3", body=query)

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])


C:\Users\yaya-\AppData\Local\Temp/ipykernel_4128/318030273.py:14: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="review3", body=query)


{'Company': 'Younited Credit', 'Customer': 'Alexis', 'Number_review': '2', 'Language': 'FR', 'Title': 'Very good service', 'Date_review': '2023-10-23', 'Reply': 'Bonjour,Nous sommes enchantés de vous lire et ravis d’avoir contribué à la réalisation de votre projet**.**Encore bienvenue chez Younited !Excellente journée,Diane', 'Date_reply': '2023-10-24', 'Rating': '5', 'Status': 'Verified', 'Experience': 'Very good service', 'Date_experience': '2023-10-18', 'Response_time': '1', 'document_id': '0'}
{'Company': 'Younited Credit', 'Customer': 'Exo Saru', 'Number_review': '3', 'Language': 'FR', 'Title': 'Super simple', 'Date_review': '2023-08-24', 'Reply': 'Bonjour,Nous vous remercions d’avoir pris le temps de nous laisser un message et sommes ravis de lire que vous êtes pleinement satisfait de votre expérience Younited.Nous vous remercions une nouvelle fois pour votre confiance.Excellente journée,Alexandra', 'Date_reply': '2023-08-25', 'Rating': '5', 'Status': 'Verified', 'Experience': 'S

In [25]:
# Requête pour obtenir les 5 mots clés positifs
es = Elasticsearch(hosts="http://localhost:9200")

# Index des avis clients
index_name = "review3"

query = {
    "query": {
        "match_all": {}
    },
    "aggs": {
        "positive_words": {
            "terms": {
                "field": "Experience",
                "size": 5,
                "min_doc_count": 5
            },
            "aggs": {
                "rating_filter": {
                    "filter": {
                        "range": {
                            "Rating": {
                                "gte": 5,
                                "lte": 5
                            }
                        }
                    }
                }
            }
        }
    }
}

# Exécution de la requête


# Exécution de la requête
result = es.search(index=index_name, body = query)

# Récupération des résultats
positive_words = result["aggregations"]["positive_words"]["buckets"]

for positive_word in positive_words:
    print(f"Mot clé positif : {positive_word['key']} - Nombre d'occurrences : {positive_word['doc_count']}")


C:\Users\yaya-\AppData\Local\Temp/ipykernel_4128/1065288932.py:38: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body = query)


Mot clé positif : Simple et rapide - Nombre d'occurrences : 52
Mot clé positif : Simple et efficace - Nombre d'occurrences : 31
Mot clé positif : Excellent service - Nombre d'occurrences : 27
Mot clé positif : Très bon service - Nombre d'occurrences : 26
Mot clé positif : Rapide et simple - Nombre d'occurrences : 20


In [26]:
es = Elasticsearch(hosts="http://localhost:9200")

# Index des avis clients
index_name = "review3"

query = {
    "query": {
        "match_all": {}
    },
    "aggs": {
        "negative_words": {
            "terms": {
                "field": "Title",
                "size": 10,
                "min_doc_count": 5
            },
            "aggs": {
                "rating_filter": {
                    "filter": {
                        "range": {
                            "Rating": {
                                "gte": 1,
                                "lte": 1
                            }
                        }
                    }
                }
            }
        }
    }
}
# Exécution de la requête
result = es.search(index=index_name, body=query)

# Récupération des résultats
negative_words = result["aggregations"]["negative_words"]["buckets"]

# Affichage des résultats
for negative_word in negative_words:
    print(f"Mot clé négatif : {negative_word['key']} - Nombre d'occurrences : {negative_word['doc_count']}")

C:\Users\yaya-\AppData\Local\Temp/ipykernel_4128/1870722822.py:33: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


Mot clé négatif : Simple et rapide - Nombre d'occurrences : 57
Mot clé négatif : Simple - Nombre d'occurrences : 49
Mot clé négatif : Excellent - Nombre d'occurrences : 39
Mot clé négatif : Simple et efficace - Nombre d'occurrences : 39
Mot clé négatif : Rapide - Nombre d'occurrences : 37
Mot clé négatif : Super - Nombre d'occurrences : 35
Mot clé négatif : Excellent service - Nombre d'occurrences : 29
Mot clé négatif : Très bon service - Nombre d'occurrences : 27
Mot clé négatif : Parfait - Nombre d'occurrences : 24
Mot clé négatif : Rapide et simple - Nombre d'occurrences : 24


# Importer Elasticsearch-py pour la gestion de l'API Elasticsearch
from elasticsearch import Elasticsearch

# Se connecter à Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Définir un normalizer de minuscules
lowercase_normalizer = {
    "settings": {
        "analysis": {
            "normalizer": {
                "lowercase_normalizer": {
                    "type": "custom",
                    "filter": ["lowercase"]
                }
            }
        }
    }
}

# Appliquer le normalizer de minuscules à un champ
mapping = {
    "mappings": {
        "properties": {
            "Experience": {
                "type": "text",
                "normalizer": "lowercase_normalizer"
            }
        }
    }
}

# Créer l'index avec le mapping et le normalizer
es.indices.create(index="index_name", body={"settings": lowercase_normalizer})
es.indices.put_mapping(index="index_name", body=mapping)


query = {
    "query": {
        "match_all": {}
    },
    "aggs": {
        "mots_negatifs": {
            "terms": {
                "field": "Experience",
                "size": 10000,
                "include": [
                    "mauvais",
                    "terrible",
                    "déçu",
                    "pauvre",
                    "insatisfait",
                    "bad",
                    "avoid",
                    "arnaque",
                    "disaster",
                    "worse",
                    "incompétent"
                ],
                "analyzer": "lowercase_analyzer"
            }
        }
    }
}

result = es.search(index=index_name, body=query)

# Récupération des résultats
mots_negatifs = result["aggregations"]["mots_negatifs"]["buckets"]

# Affichage des résultats
for mot_negatif in mots_negatifs:
    print(f"Mot négatif : {mot_negatif['key']} - Nombre d'occurrences : {mot_negatif['doc_count']}")


In [27]:
es = Elasticsearch(hosts="http://localhost:9200")

# Index des avis clients
index_name = "review3"

# Requête d'agrégation pour obtenir des statistiques sur le champ "Rating"
query = {
    "size": 0,
    "aggs": {
        "value_count": {
            "value_count": {
                "field": "Rating"
            }
        },
        "rating_stats": {
            "stats": {
                "field": "Rating"
            }
        },
        "extended_ratings_stats": {
            "extended_stats": {
                "field": "Rating"
            }
        }
    }
}

# Exécuter la requête
result = es.search(index=index_name, body=query)

# Afficher les résultats de l'agrégation
aggregations = result.get("aggregations", {})
value_count = aggregations.get("value_count", {})
rating_stats = aggregations.get("rating_stats", {})
extended_ratings_stats = aggregations.get("extended_ratings_stats", {})

print("Nombre total de valeurs de rating:", value_count.get("value", 0))
print("Statistiques de rating:")
print("    Minimum:", rating_stats.get("min", 0))
print("    Maximum:", rating_stats.get("max", 0))
print("    Moyenne:", rating_stats.get("avg", 0))
print("    Somme:", rating_stats.get("sum", 0))
print("    Écart-type:", extended_ratings_stats.get("std_deviation", 0))


C:\Users\yaya-\AppData\Local\Temp/ipykernel_4128/2884975705.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


Nombre total de valeurs de rating: 2515
Statistiques de rating:
    Minimum: 1.0
    Maximum: 5.0
    Moyenne: 4.6870775347912526
    Somme: 11788.0
    Écart-type: 0.8582959723170738


In [28]:
# Liste des entreprises avec le plus de commentaires négatifs
es = Elasticsearch(hosts="http://localhost:9200")

# Index des avis clients
index_name = "review3"

# Exécution de la requête
result = es.search(index=index_name, 
body={
    "query": {
        "match_all": {}
    },
    "sort": [
        {
            "Rating": {
                "order": "asc"  # Tri ascendant (du plus bas au plus haut)
            }
        }
    ]
}
)
# Exécution de la requête pour récupérer les commentaires triés par note
#result = es.search(index=index_name, body=query)

# Analyse des commentaires triés pour identifier les entreprises avec le plus de commentaires négatifs
from collections import Counter

negative_reviews = []
for hit in result["hits"]["hits"]:
    rating = int(hit["_source"]["Rating"])  # Convertir la note en entier
    if rating < 3:  # Supposons que 1 à 2 sont considérés comme des notes négatives
        negative_reviews.append(hit["_source"]["Company"])

# Utilisation de Counter pour compter le nombre de commentaires négatifs par entreprise
company_count = Counter(negative_reviews)

# Triez les entreprises par le nombre de commentaires négatifs (du plus au moins)
sorted_companies = sorted(company_count.items(), key=lambda x: x[1], reverse=True)

# Afficher les entreprises avec le plus de commentaires négatifs
for company, count in sorted_companies[:6]:  # Affiche les 5 premières entreprises avec le plus de commentaires négatifs
    print(f"Entreprise : {company}, Nombre de commentaires négatifs : {count}")


C:\Users\yaya-\AppData\Local\Temp/ipykernel_4128/2648896791.py:8: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name,


Entreprise : Younited Credit, Nombre de commentaires négatifs : 7
Entreprise : Cofidis, Nombre de commentaires négatifs : 3


In [29]:
es = Elasticsearch(hosts="http://localhost:9200")

# Index des avis clients
index_name = "review3"

# Exécution de la requête
result = es.search(index=index_name, 
body={
    "query": {
        "match": {
            "Reply": {
                "query": "No Reply"
            }
        }
    },
    "aggs": {
        "companies": {
            "terms": {
                "field": "Company",
                "size": 10
            }
        }
    }
})

# Récupération des résultats
companies = result["aggregations"]["companies"]["buckets"]

# Affichage des résultats
for company in companies:
    print(f"Entreprise : {company['key']} - Nombre de No Reply : {company['doc_count']}")

C:\Users\yaya-\AppData\Local\Temp/ipykernel_4128/2866378900.py:7: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name,


Entreprise : BoursoBank - Nombre de No Reply : 429
Entreprise : Younited Credit - Nombre de No Reply : 414
Entreprise : Orange Bank - Nombre de No Reply : 87
Entreprise : Floabank - Nombre de No Reply : 41
Entreprise : Anytime - Nombre de No Reply : 25
Entreprise : Cofidis - Nombre de No Reply : 1


In [30]:
# Exécution de la requête pour obtenir le classement des entreprises selon leur note moyenne
query = {
    "aggs": {
        "companies": {
            "terms": {
                "field": "Company"
            },
            "aggs": {
                "average_rating": {
                    "avg": {
                        "field": "Rating"
                    }
                }
            }
        }
    }
}

result = es.search(index=index_name, body=query)

# Récupération des résultats
companies = result["aggregations"]["companies"]["buckets"]

# Tri des entreprises par leur note moyenne
companies.sort(key=lambda company: company["average_rating"]["value"], reverse=False)

# Affichage des résultats
for company in companies:
    average_ratings = round(company['average_rating']['value'], 2)
    print(f"Entreprise : {company['key']} - Note moyenne : {average_ratings}")


Entreprise : Floabank - Note moyenne : 4.49
Entreprise : Orange Bank - Note moyenne : 4.6
Entreprise : Anytime - Note moyenne : 4.63
Entreprise : BoursoBank - Note moyenne : 4.64
Entreprise : Cofidis - Note moyenne : 4.8
Entreprise : Younited Credit - Note moyenne : 4.81


C:\Users\yaya-\AppData\Local\Temp/ipykernel_4128/4117333319.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


In [31]:
# Exécution de la requête pour obtenir le groupe selon leur statut
query = {
    "aggs": {
        "statuses": {
            "terms": {
                "field": "Status"
            }
        }
    }
}

result = es.search(index=index_name, body=query)

# Récupération des résultats
statuses = result["aggregations"]["statuses"]["buckets"]

# Affichage des résultats
for status in statuses:
    print(f"Status : {status['key']} - Nombre de commentateurs : {status['doc_count']}")


Status : Invited - Nombre de commentateurs : 1533
Status : Verified - Nombre de commentateurs : 545
Status : Merged - Nombre de commentateurs : 417
Status : Redirected - Nombre de commentateurs : 20


C:\Users\yaya-\AppData\Local\Temp/ipykernel_4128/3781268496.py:12: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


In [32]:
# Exécution de la requête
query = {
    "query": {
        "match": {
            "Reply": "No Reply"
        }
    },
    "aggs": {
        "total_no_reply": {
            "value_count": {
                "field": "Reply"
            }
        },
        "companies": {
            "terms": {
                "field": "Company"
            },
            "aggs": {
                "number_of_no_reply": {
                    "value_count": {
                        "field": "Reply"
                    }
                }
            }
        }
    }
}

result = es.search(index=index_name, body=query)
# Récupération des résultats
total_no_reply = result["aggregations"]["total_no_reply"]["value"]
companies = result["aggregations"]["companies"]["buckets"]

# Affichage des résultats
for company in companies:
    number_of_no_reply = company["number_of_no_reply"]["value"]
    percentage = (number_of_no_reply / total_no_reply) * 100
    print(f"Entreprise : {company['key']} - Nombre de No Reply : {number_of_no_reply} - Taux de No Reply : {percentage:.2f}%")


Entreprise : BoursoBank - Nombre de No Reply : 429 - Taux de No Reply : 43.03%
Entreprise : Younited Credit - Nombre de No Reply : 414 - Taux de No Reply : 41.52%
Entreprise : Orange Bank - Nombre de No Reply : 87 - Taux de No Reply : 8.73%
Entreprise : Floabank - Nombre de No Reply : 41 - Taux de No Reply : 4.11%
Entreprise : Anytime - Nombre de No Reply : 25 - Taux de No Reply : 2.51%
Entreprise : Cofidis - Nombre de No Reply : 1 - Taux de No Reply : 0.10%


C:\Users\yaya-\AppData\Local\Temp/ipykernel_4128/3237135795.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


In [34]:
# Exécution de la requête pour obtenir le classement des entreprises selon leur note moyenne
query = {
    "aggs": {
        "companies": {
            "terms": {
                "field": "Company"
            },
            "aggs": {
                "average_day": {
                    "avg": {
                        "field": "Response_time"
                    }
                }
            }
        }
    }
}

result = es.search(index=index_name, body=query)

# Récupération des résultats
companies = result["aggregations"]["companies"]["buckets"]

# Tri des entreprises par leur note moyenne
companies.sort(key=lambda company: company["average_day"]["value"], reverse=False)

# Affichage des résultats
for company in companies:
    average_days = round(company['average_day']['value'], 2)
    print(f"Entreprise : {company['key']} - Délai de réponse moyen : {average_days}")


Entreprise : BoursoBank - Délai de réponse moyen : 0.0
Entreprise : Floabank - Délai de réponse moyen : 0.62
Entreprise : Younited Credit - Délai de réponse moyen : 14.38
Entreprise : Cofidis - Délai de réponse moyen : 15.25
Entreprise : Orange Bank - Délai de réponse moyen : 26.1
Entreprise : Anytime - Délai de réponse moyen : 34.98


C:\Users\yaya-\AppData\Local\Temp/ipykernel_4128/2153710635.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)
